In [1]:
!pip install line_profiler


You are using pip version 9.0.1, however version 24.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [2]:
%load_ext line_profiler

In [5]:
def sum_of_lists(N) :
    total = 0
    for i in range(5) :
    #List Comphrehension in for loop
        L = [j^ (j >> 1) for j in range (N)] 
        total += sum(L)
    return total
                

In [8]:
#line by line Profiling -provides the time in milli seconds
%lprun -f sum_of_lists sum_of_lists(5000)

4 function calls in 0.696 seconds

   Ordered by: internal time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        5    0.550    0.110    0.550    0.110 <ipython-input-5-e36250cf8b7a>:5(<listcomp>)
        5    0.099    0.020    0.099    0.020 {built-in method builtins.sum}
        1    0.038    0.038    0.687    0.687 <ipython-input-5-e36250cf8b7a>:1(sum_of_lists)
        1    0.010    0.010    0.696    0.696 <string>:1(<module>)
        1    0.000    0.000    0.696    0.696 {built-in method builtins.exec}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}

In [21]:
#prun to find profile results
%prun sum_of_lists(1000000)

#memory_profiler-> to check the memory usage

In [9]:
!pip install memory_profiler


You are using pip version 9.0.1, however version 24.3.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [10]:
%load_ext memory_profiler

In [11]:
%memit sum_of_lists(1000000)  #memory usage in MB

peak memory: 136.75 MiB, increment: 68.46 MiB


For a line-by-line description of memory use, we can use the %mprun magic. 
Unfortunately, this magic works only for functions defined in separate modules rather than the notebook itself, 
so we'll start by using the %%file magic to create a simple module called mprun_demo.py, 
which contains our sum_of_lists function, with one addition that will make our memory profiling results more clear:


In [15]:
%%file mprun_demo.py
def sum_of_list(N) :
    total =0 
    for i in range(5) : 
        L = [j^ (j >> 1) for j in range (N)] 
        total += sum(L)
        del L ##Remove Ref to L
    return total    


Overwriting mprun_demo.py


We can now import the new version of this function and run the memory line profiler:



In [18]:
from mprun_demo  import sum_of_list
%mprun -f sum_of_list sum_of_list(100000)

Filename: D:\Data_Science\Data_Science\Matplotlib\mprun_demo.py

Line #    Mem usage    Increment  Occurrences   Line Contents
=============================================================
     1     69.9 MiB     69.9 MiB           1   def sum_of_list(N) :
     2     69.9 MiB      0.0 MiB           1       total =0 
     3     69.9 MiB      0.0 MiB           6       for i in range(5) : 
     4     72.7 MiB -615987.5 MiB      500015           L = [j^ (j >> 1) for j in range (N)] 
     5     72.7 MiB      0.0 MiB           5           total += sum(L)
     6     69.9 MiB    -14.4 MiB           5           del L ##Remove Ref to L
     7     69.9 MiB      0.0 MiB           1       return total

Here the Increment column tells us how much each line affects the total memory budget: observe that when we create and delete the list L, we are adding about 25 MB of memory usage. This is on top of the background memory usage from the Python interpreter itself.